In [1]:
#pragma cling add_library_path("/home/abhishek/Downloads/libtorch/lib/")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch/include")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch/include/torch/csrc/api/include/")

In [2]:
#pragma cling load("/home/abhishek/Downloads/libtorch/lib/libc10.so")
#pragma cling load("/home/abhishek/Downloads/libtorch/lib/libgomp-75eea7e8.so.1")
#pragma cling load("/home/abhishek/Downloads/libtorch/lib/libtorch.so")

In [3]:
#include "Main.h"

In [7]:
std::cout << torch::randn({3, 2});

 0.2164 -1.2534
 0.2482 -1.3430
 0.0711  0.9113
[ CPUFloatType{3,2} ]

## Training function

In [5]:
template <typename Dataloader>
void train(
    torch::jit::script::Module net,
    torch::nn::Linear lin,
    Dataloader &train_dl,
    torch::optim::Optimizer &optimizer,
    size_t dataset_size,
    torch::Device device)
{
    // To calculate loss and accuracy per epoch
    float avg_loss = 0.0f;
    int64_t avg_accuracy = 0.0f;

    // Put model and lin to (device)
    net.to(device);
    lin->to(device);

    int64_t batch_index = 0;

    for (auto &batch : train_dl)
    {
        // Zero the gradients of optimizer
        optimizer.zero_grad();

        // Fetch features and labels for current batch
        auto data = batch.data.to(device);
        auto targets = batch.target.to(device);

        /* 
        Make a vector of jit Ivalues 
        and push `data` into it,
        to be able to pass it through the jit model 
        */
        std::vector<torch::jit::IValue> input;
        input.push_back(data);

        // Get the output from the jit model and cast into tensor
        auto output = net.forward(input).toTensor();

        // Resize output tesnor
        output = output.view({output.size(0), -1});

        // pass through linear layer for final output
        output = lin(output);

        // Calculate the loss
        auto loss = torch::nn::CrossEntropyLoss()(output, targets);

        if (++batch_index % Main().log_interval == 0)
            std::printf(
                "\r Batch: %ld Loss: %.4f",
                ++batch_index,
                loss.template item<float>());
        /* 
        Usual PyTorch steps:
            backprop on the nll_loss
            step the optimizer 
        */
        loss.backward();
        optimizer.zero_grad();

        //Calculate number of corrects
        auto corrects = output.argmax(1).eq(targets).sum();

        avg_loss += loss.template item<float>();
        avg_accuracy += corrects.template item<int64_t>();
    }

    std::cout << "\n Average Training loss: " << avg_loss / float(dataset_size);
    std::cout << "\n Average Training accuracy: " << (avg_accuracy / float(dataset_size)) * 100 << "%";
}


## Testing function

In [6]:
template <typename Dataloader>
void test(
    torch::jit::script::Module net,
    torch::nn::Linear lin,
    Dataloader &test_dl,
    size_t dataset_size,
    float best_loss)
{
    // To calculate loss and accuracy per epoch
    float avg_loss = 0.0f;
    int64_t avg_accuracy = 0;

    int64_t batch_index = 0;

    net.to(torch::kCPU);
    lin->to(torch::kCPU);

    for (auto &batch : test_dl)
    {
        // Fetch features and labels for current batch
        auto data = batch.data;
        auto targets = batch.target;

        /* 
        Make a vector of jit Ivalues 
        and push `data` into it,
        to be able to pass it through the jit model 
        */
        std::vector<torch::jit::IValue> input;
        input.push_back(data);

        // Get the output from the jit model and cast into tensor
        auto output = net.forward(input).toTensor();

        // Resize output tesnor
        output = output.view({output.size(0), -1});

        // pass through linear layer for final output
        output = lin(output);

        // Calculate the loss
        auto loss = torch::nn::CrossEntropyLoss()(output, targets);

        if (++batch_index % Main().log_interval == 0)
            std::printf(
                "\r Batch: %ld Loss: %.4f",
                ++batch_index,
                loss.template item<float>());

        //Calculate number of corrects
        auto corrects = output.argmax(1).eq(targets).sum();

        avg_loss += loss.template item<float>();
        avg_accuracy += corrects.template item<int64_t>();
    }

    float final_loss = avg_loss / float(dataset_size);

    std::cout << "\n Average Testing loss: " << final_loss;
    std::cout << "\n Average Testing accuracy: " << (avg_accuracy / float(dataset_size)) * 100 << "%";

    if(final_loss < best_loss ){
        std::cout << "\n Saving model..." << std::endl;
        torch::save(lin, "best_model.pt");
    }
}